In [1]:
import requests
from bs4 import BeautifulSoup as bs

In [17]:
url ='https://cdaweb.gsfc.nasa.gov/pub/data/wind/swe/swe_h1/2004'

dataset_response = requests.get(url)
name_list = [i.text for i in bs(dataset_response.content).find_all('td') if i.text[-3:] =='cdf']

In [18]:
len(name_list)

364

In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor



#len(name_list) =365

year  = '2002'
def download_file(name):
    try:
        response = requests.get(f'https://cdaweb.gsfc.nasa.gov/pub/data/wind/swe/swe_h1/{year}/{name}')
        with open(f'down_test\\{year}\\{name}', 'wb') as fp:
            fp.write(response.content)
        
        print(f"Downloaded: {name}")
    except Exception as e:
        print(f"Error downloading {name}: {e}")

def download_files_with_threadpool(name_list):
    with ThreadPoolExecutor() as executor:
        executor.map(download_file, name_list)

download_files_with_threadpool(name_list)


In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor
import math

year = '2004'
MAX_RETRIES = 3  
def download_file(name):
    retries = 0
    while retries < MAX_RETRIES:
        try:
            response = requests.get(f'https://cdaweb.gsfc.nasa.gov/pub/data/wind/swe/swe_h1/{year}/{name}')
            if response.status_code == 200:
                with open(f'down_test\\{year}\\{name}', 'wb') as fp:
                    fp.write(response.content)
                print(f"Downloaded: {name}")

                break
            else:
                print(f"Error downloading {name}: Got status code {response.status_code}")
        except Exception as e:
            print(f"Error downloading {name}: {e}")
        
        retries += 1
        print(f"Retrying ({retries}/{MAX_RETRIES})...")
    


progress = len(name_list)
def download_files_with_threadpool(name_list, max_workers=10):
    chunk_size = math.ceil(len(name_list) / max_workers)
    chunks = [name_list[i:i+chunk_size] for i in range(0, len(name_list), chunk_size)]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for chunk in chunks:
            executor.map(download_file, chunk)


download_files_with_threadpool(name_list, max_workers=10)
